In [5]:
!pip install git+https://github.com/huggingface/parler-tts.git


  Cloning https://github.com/huggingface/parler-tts.git to /tmp/pip-req-build-ew8vc3nx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/parler-tts.git /tmp/pip-req-build-ew8vc3nx
  Resolved https://github.com/huggingface/parler-tts.git to commit d108732cd57788ec86bc857d99a6cabd66663d68
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/descriptinc/audiotools to /tmp/pip-install-l5lcwtgw/descript-audiotools_809a5545847f4cce93690e0b5ffb1ec2
  Running command git clone --filter=blob:none --quiet https://github.com/descriptinc/audiotools /tmp/pip-install-l5lcwtgw/descript-audiotools_809a5545847f4cce93690e0b5ffb1ec2
  Resolved https://github.com/descriptinc/audiotools to commit 348ebf2034ce24e2a91a553e3171cb00c0c71678
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
  Prep

In [2]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

# Device setup
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Using:", device)

# Load model & tokenizers
model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
description_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

# Prompt & description
prompt = "Hey, how are you doing today?"
description = (
    "A female speaker with a British accent delivers a slightly expressive "
    "and animated speech with a moderate speed and pitch. The recording is "
    "of very high quality, with the speaker's voice sounding clear and very close up."
)

# Tokenize & move to device properly
description_inputs = description_tokenizer(description, return_tensors="pt")
prompt_inputs = tokenizer(prompt, return_tensors="pt")

description_inputs = {k: v.to(device) for k, v in description_inputs.items()}
prompt_inputs = {k: v.to(device) for k, v in prompt_inputs.items()}

# Generate audio
generation = model.generate(
    input_ids=description_inputs["input_ids"],
    attention_mask=description_inputs["attention_mask"],
    prompt_input_ids=prompt_inputs["input_ids"],
    prompt_attention_mask=prompt_inputs["attention_mask"]
)

# Convert to numpy and save
audio_arr = generation.cpu().numpy().squeeze()
sf.write("indic_tts_out.wav", audio_arr, model.config.sampling_rate)

print("✅ Audio saved as indic_tts_out.wav")

from IPython.display import Audio
Audio("indic_tts_out.wav")



Using: cuda:0


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.75G [00:00<?, ?B/s]

  "_name_or_path": "google/flan-t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.46.1",
  "use_cache": true,
  "vocab_size": 32128
}

  "_name_or_path": "ylacombe/dac_44khz",
  "architectures": [
    "DacModel"
  ],
  "codebook_dim": 8,
  "codebook_loss_weight": 1.0,
  "codebook_size": 1024,
  "commitment_loss_weight": 0.25,
  "decoder_hidden_si

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

✅ Audio saved as indic_tts_out.wav


Method 2


In [1]:
from google.colab import files
uploaded = files.upload()   # select your .srt file from local computer
srt_file = list(uploaded.keys())[0]
print("Uploaded:", srt_file)

Saving 2F.srt to 2F (3).srt
Uploaded: 2F (3).srt


In [3]:
import re
import torch
import soundfile as sf
import numpy as np
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer

#setup
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Using:", device)

#Load model + tokenizers
model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
description_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

# Voice description (you can tweak style here)
description = "A female Indian English speaker with a clear, natural, and steady voice, moderate pitch and pace."
desc_inputs = description_tokenizer(description, return_tensors="pt")
desc_inputs = {k: v.to(device) for k, v in desc_inputs.items()}

#Parse SRT file → subtitle text lines
def parse_srt(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        content = f.read()

    blocks = re.split(r"\n\s*\n", content.strip())
    texts = []
    for block in blocks:
        lines = block.strip().split("\n")
        if len(lines) >= 3:
            subtitle_text = " ".join(lines[2:])  # join multi-line subtitles
            texts.append(subtitle_text)
    return texts

texts = parse_srt(srt_file)
print("Parsed", len(texts), "subtitle segments")

#Generate audio for all subtitles and merge into one
all_audio = []

for i, line in enumerate(texts, start=1):
    if not line.strip():
        continue
    # Tokenize
    prompt_inputs = tokenizer(line, return_tensors="pt")
    prompt_inputs = {k: v.to(device) for k, v in prompt_inputs.items()}
    # Generate audio
    generation = model.generate(
        input_ids=desc_inputs["input_ids"],
        attention_mask=desc_inputs["attention_mask"],
        prompt_input_ids=prompt_inputs["input_ids"],
        prompt_attention_mask=prompt_inputs["attention_mask"]
    )
    audio = generation.cpu().numpy().squeeze()
    all_audio.append(audio)
    print(f"Generated segment {i}/{len(texts)}")

# Concatenate into one narration
merged_audio = np.concatenate(all_audio)
out_file = "final_narration.wav"
sf.write(out_file, merged_audio, model.config.sampling_rate)
print("✅ Saved:", out_file)

#Play audio in Colab
from IPython.display import Audio
Audio(out_file)


Output hidden; open in https://colab.research.google.com to view.

Method 3 : uses Stretching



In [4]:

import re
import librosa
import numpy as np
import torch
import soundfile as sf
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
from google.colab import files



# ---- Device ----
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Using:", device)

# ---- Load model ----
model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
description_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

# ---- Fixed voice description ----
description = (
    "A clear Indian female speaker with an educational tone, "
    "moderate pace, steady rhythm, and neutral pitch. "
    "The recording is natural and classroom-like."
)
desc_inputs = description_tokenizer(description, return_tensors="pt")
desc_inputs = {k: v.to(device) for k, v in desc_inputs.items()}

# ---- Parse SRT with timestamps ----
def parse_srt(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        content = f.read()

    blocks = re.split(r"\n\s*\n", content.strip())
    segments = []
    for block in blocks:
        lines = block.strip().split("\n")
        if len(lines) >= 3:
            # Extract times
            times = lines[1]
            start, end = times.split(" --> ")

            def to_sec(t):
                h, m, s_ms = t.split(":")
                s, ms = s_ms.split(",")
                return int(h)*3600 + int(m)*60 + int(s) + int(ms)/1000.0

            start_s = to_sec(start.strip())
            end_s = to_sec(end.strip())
            duration = end_s - start_s

            # Subtitle text
            text = " ".join(lines[2:])
            segments.append((start_s, end_s, duration, text))
    return segments


segments = parse_srt(srt_file)
print("Parsed", len(segments), "segments")

# ---- Generate & time-stretch audio ----
all_audio = []

for i, (start, end, target_dur, line) in enumerate(segments, start=1):
    if not line.strip():
        continue
    # Tokenize
    prompt_inputs = tokenizer(line, return_tensors="pt")
    prompt_inputs = {k: v.to(device) for k, v in prompt_inputs.items()}
    # Generate speech
    generation = model.generate(
        input_ids=desc_inputs["input_ids"],
        attention_mask=desc_inputs["attention_mask"],
        prompt_input_ids=prompt_inputs["input_ids"],
        prompt_attention_mask=prompt_inputs["attention_mask"]
    )
    audio = generation.cpu().numpy().squeeze()

    # Adjust duration to match subtitle
    orig_dur = len(audio) / model.config.sampling_rate
    if orig_dur > 0:
        stretch_factor = target_dur / orig_dur
        if stretch_factor != 1.0:
            audio = librosa.effects.time_stretch(audio.astype(np.float32), rate=stretch_factor)

    all_audio.append(audio)
    print(f"Segment {i}: target={target_dur:.2f}s, generated={orig_dur:.2f}s, stretched={len(audio)/model.config.sampling_rate:.2f}s")

# ---- Merge all ----
merged_audio = np.concatenate(all_audio)
out_file = "final_matched_narration.wav"
sf.write(out_file, merged_audio, model.config.sampling_rate)
print("✅ Saved with timing:", out_file)

# ---- Play back ----
from IPython.display import Audio
Audio(out_file)


Output hidden; open in https://colab.research.google.com to view.

Method 3: Silence padding

In [2]:

import re
import librosa
import numpy as np
import torch
import soundfile as sf
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
from google.colab import files



# ---- Device ----
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Using:", device)

# ---- Load model ----
model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
description_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

# ---- Fixed voice description ----
description = (
    "A clear Indian female speaker with an educational tone, "
    "moderate pace, steady rhythm, and neutral pitch. "
    "The recording is natural and classroom-like."
)
desc_inputs = description_tokenizer(description, return_tensors="pt")
desc_inputs = {k: v.to(device) for k, v in desc_inputs.items()}

# ---- Parse SRT with timestamps ----
def parse_srt(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        content = f.read()

    blocks = re.split(r"\n\s*\n", content.strip())
    segments = []
    for block in blocks:
        lines = block.strip().split("\n")
        if len(lines) >= 3:
            # Extract times
            times = lines[1]
            start, end = times.split(" --> ")

            def to_sec(t):
                h, m, s_ms = t.split(":")
                s, ms = s_ms.split(",")
                return int(h)*3600 + int(m)*60 + int(s) + int(ms)/1000.0

            start_s = to_sec(start.strip())
            end_s = to_sec(end.strip())
            duration = end_s - start_s

            # Subtitle text
            text = " ".join(lines[2:])
            segments.append((start_s, end_s, duration, text))
    return segments


segments = parse_srt(srt_file)
print("Parsed", len(segments), "segments")

# ---- Generate & time-stretch audio ----
all_audio = []
sr = model.config.sampling_rate

for i, (start, end, target_dur, line) in enumerate(segments, start=1):
    if not line.strip():
        continue

    # Tokenize
    prompt_inputs = tokenizer(line, return_tensors="pt")
    prompt_inputs = {k: v.to(device) for k, v in prompt_inputs.items()}

    # Generate audio
    generation = model.generate(
        input_ids=desc_inputs["input_ids"],
        attention_mask=desc_inputs["attention_mask"],
        prompt_input_ids=prompt_inputs["input_ids"],
        prompt_attention_mask=prompt_inputs["attention_mask"]
    )
    audio = generation.cpu().numpy().squeeze()
    orig_dur = len(audio) / sr

    # --- Instead of stretching, pad/truncate ---
    if orig_dur < target_dur:
        # Add silence to match subtitle duration
        silence = np.zeros(int((target_dur - orig_dur) * sr))
        audio = np.concatenate([audio, silence])
    elif orig_dur > target_dur:
        # Optionally truncate if it's too long
        audio = audio[:int(target_dur * sr)]

    all_audio.append(audio)
    print(f"Segment {i}: target={target_dur:.2f}s, generated={orig_dur:.2f}s, final={len(audio)/sr:.2f}s")

# ---- Merge all ----
merged_audio = np.concatenate(all_audio)
out_file = "final_matched_narration.wav"
sf.write(out_file, merged_audio, model.config.sampling_rate)
print("✅ Saved with timing:", out_file)

# ---- Play back ----
from IPython.display import Audio
Audio(out_file)


Output hidden; open in https://colab.research.google.com to view.

Method 4 : Mixed Speaker problem .Sol

In [4]:
!pip install pysrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=f4ffe61f6b75084e9610e1ff0537bd4395147519fdebd658afe56780c590dd4a
  Stored in directory: /root/.cache/pip/wheels/6a/36/54/2aa8dc961885dfa7b0ebd45a57505f25039d79b4ea0fd9f29d
Successfully built pysrt


In [5]:
import re
import numpy as np
import soundfile as sf
import torch
import pysrt
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
from google.colab import files

# -----------------------
# Upload your .srt file
# -----------------------
uploaded = files.upload()
srt_file = list(uploaded.keys())[0]

# -----------------------
# Parse .srt file
# -----------------------
def parse_srt(path):
    subs = pysrt.open(path)
    segments = []
    for sub in subs:
        start_s = sub.start.hours*3600 + sub.start.minutes*60 + sub.start.seconds + sub.start.milliseconds/1000
        end_s   = sub.end.hours*3600   + sub.end.minutes*60   + sub.end.seconds   + sub.end.milliseconds/1000
        text    = sub.text.replace("\n", " ").strip()
        segments.append((start_s, end_s, text))
    return segments

segments = parse_srt(srt_file)
print("✅ Parsed", len(segments), "segments from", srt_file)

# -----------------------
# Load model + tokenizer
# -----------------------
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
desc_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

# 🔒 Fix speaker (Swapna)
description = "Swapna, an Indian female speaker with an educational and clear tone."
desc_inputs = desc_tokenizer(description, return_tensors="pt").to(device)

sr = model.config.sampling_rate
full_audio = []
current_time = 0.0

# -----------------------
# Generate audio per segment
# -----------------------
for i, (start, end, text) in enumerate(segments):
    print(f"🔊 Generating segment {i+1}/{len(segments)}: {text[:60]}...")

    # Insert silence if there's a gap before this subtitle
    gap = start - current_time
    if gap > 0:
        full_audio.append(np.zeros(int(gap * sr)))

    # TTS generation
    text_inputs = tokenizer(text, return_tensors="pt").to(device)
    gen = model.generate(
        input_ids=desc_inputs["input_ids"],
        attention_mask=desc_inputs["attention_mask"],
        prompt_input_ids=text_inputs["input_ids"],
        prompt_attention_mask=text_inputs["attention_mask"]
    )
    audio = gen.cpu().numpy().squeeze()

    # Stretch/compress to fit subtitle duration
    duration = end - start
    actual_dur = len(audio) / sr
    if actual_dur > 0:
        factor = duration / actual_dur
        import librosa
        audio = librosa.effects.time_stretch(audio.astype(np.float32), rate=factor)

    full_audio.append(audio)
    current_time = end

# -----------------------
# Merge + save
# -----------------------
merged = np.concatenate(full_audio)
sf.write("swapna_srt_output.wav", merged, sr)
print("✅ Final audio saved as swapna_srt_output.wav")

# ---- Play back ----
from IPython.display import Audio
Audio(out_file)


Output hidden; open in https://colab.research.google.com to view.

Method 5: mutilang+indic autopick

In [7]:
import re
import numpy as np
import soundfile as sf
import torch
import pysrt
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
from google.colab import files

# Step 1: Upload .srt file
uploaded = files.upload()
srt_file = list(uploaded.keys())[0]

# Step 2: Parse .srt into segments with timings
def parse_srt(path):
    subs = pysrt.open(path)
    segments = []
    for sub in subs:
        start = sub.start.hours*3600 + sub.start.minutes*60 + sub.start.seconds + sub.start.milliseconds/1000
        end   = sub.end.hours*3600   + sub.end.minutes*60   + sub.end.seconds   + sub.end.milliseconds/1000
        text = sub.text.replace("\n", " ").strip()
        segments.append((start, end, text))
    return segments

segments = parse_srt(srt_file)
print("Parsed", len(segments), "segments.")

# Step 3: Load model and tokenizers
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
desc_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

# Step 4: Prepare speaker descriptions
speaker_prompts = {
    "english":  "Swapna, an Indian female speaker with a clear educational tone.",
    "hindi":    "Divya, an Indian female speaker with a clear educational tone.",
    "telugu":   "Lalitha, an Indian female speaker with a clear educational tone."
}
desc_inputs_dict = {
    lang: desc_tokenizer(prompt, return_tensors="pt").to(device)
    for lang, prompt in speaker_prompts.items()
}

sr = model.config.sampling_rate
full_audio = []
current_time = 0.0

# Step 5: Generate audio per segment, choosing speaker by detected language
for idx, (start, end, text) in enumerate(segments, start=1):
    # Detect language: basic heuristic (presence of Devanagari or Telugu script)
    if re.search(r'[\u0C00-\u0C7F]', text):
        lang = "telugu"
    elif re.search(r'[\u0900-\u097F]', text):
        lang = "hindi"
    else:
        lang = "english"
    desc_inputs = desc_inputs_dict[lang]

    print(f"{idx}/{len(segments)} | {lang.capitalize()} | Speaker: {speaker_prompts[lang].split(',')[0]} → \"{text[:50]}...\"")

    # Add silence if there's a gap before this subtitle
    gap = start - current_time
    if gap > 0:
        full_audio.append(np.zeros(int(gap * sr)))

    # Generate speech
    text_inputs = tokenizer(text, return_tensors="pt").to(device)
    gen = model.generate(
        input_ids=desc_inputs["input_ids"],
        attention_mask=desc_inputs["attention_mask"],
        prompt_input_ids=text_inputs["input_ids"],
        prompt_attention_mask=text_inputs["attention_mask"]
    )
    audio = gen.cpu().numpy().squeeze()

    # Time adjustment: pad if shorter, truncate if longer
    duration = end - start
    actual_dur = len(audio) / sr
    if actual_dur < duration:
        pad = np.zeros(int((duration - actual_dur) * sr))
        audio = np.concatenate([audio, pad])
    else:
        audio = audio[:int(duration * sr)]

    full_audio.append(audio)
    current_time = end

# Step 6: Merge and save final audio
merged = np.concatenate(full_audio)
sf.write("multilang_swapna_divya_lalitha.wav", merged, sr)
print("Saved:", "multilang_swapna_divya_lalitha.wav")

from IPython.display import Audio
Audio("multilang_swapna_divya_lalitha.wav")


Output hidden; open in https://colab.research.google.com to view.

method 5 test 2


In [8]:
import re
import numpy as np
import soundfile as sf
import torch
import pysrt
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
from google.colab import files

# Step 1: Upload .srt file
uploaded = files.upload()
srt_file = list(uploaded.keys())[0]

# Step 2: Parse .srt into segments with timings
def parse_srt(path):
    subs = pysrt.open(path)
    segments = []
    for sub in subs:
        start = sub.start.hours*3600 + sub.start.minutes*60 + sub.start.seconds + sub.start.milliseconds/1000
        end   = sub.end.hours*3600   + sub.end.minutes*60   + sub.end.seconds   + sub.end.milliseconds/1000
        text = sub.text.replace("\n", " ").strip()
        segments.append((start, end, text))
    return segments

segments = parse_srt(srt_file)
print("Parsed", len(segments), "segments.")

# Step 3: Load model and tokenizers
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
desc_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

# Step 4: Prepare speaker descriptions
speaker_prompts = {
    "english":  "Swapna, an Indian female speaker with a clear educational tone.",
    "hindi":    "Divya, an Indian female speaker with a clear educational tone.",
    "telugu":   "Lalitha, an Indian female speaker with a clear educational tone."
}
desc_inputs_dict = {
    lang: desc_tokenizer(prompt, return_tensors="pt").to(device)
    for lang, prompt in speaker_prompts.items()
}

sr = model.config.sampling_rate
full_audio = []
current_time = 0.0

# Step 5: Generate audio per segment, choosing speaker by detected language
for idx, (start, end, text) in enumerate(segments, start=1):
    # Detect language: basic heuristic (presence of Devanagari or Telugu script)
    if re.search(r'[\u0C00-\u0C7F]', text):
        lang = "telugu"
    elif re.search(r'[\u0900-\u097F]', text):
        lang = "hindi"
    else:
        lang = "english"
    desc_inputs = desc_inputs_dict[lang]

    print(f"{idx}/{len(segments)} | {lang.capitalize()} | Speaker: {speaker_prompts[lang].split(',')[0]} → \"{text[:50]}...\"")

    # Add silence if there's a gap before this subtitle
    gap = start - current_time
    if gap > 0:
        full_audio.append(np.zeros(int(gap * sr)))

    # Generate speech
    text_inputs = tokenizer(text, return_tensors="pt").to(device)
    gen = model.generate(
        input_ids=desc_inputs["input_ids"],
        attention_mask=desc_inputs["attention_mask"],
        prompt_input_ids=text_inputs["input_ids"],
        prompt_attention_mask=text_inputs["attention_mask"]
    )
    audio = gen.cpu().numpy().squeeze()

    # Time adjustment: pad if shorter, truncate if longer
    duration = end - start
    actual_dur = len(audio) / sr
    if actual_dur < duration:
        pad = np.zeros(int((duration - actual_dur) * sr))
        audio = np.concatenate([audio, pad])
    else:
        audio = audio[:int(duration * sr)]

    full_audio.append(audio)
    current_time = end

# Step 6: Merge and save final audio
merged = np.concatenate(full_audio)
sf.write("multilang_swapna_divya_lalitha.wav", merged, sr)
print("Saved:", "multilang_swapna_divya_lalitha.wav")

from IPython.display import Audio
Audio("multilang_swapna_divya_lalitha.wav")


Output hidden; open in https://colab.research.google.com to view.